# Calculando o Beta de um Ativo

## 1.0 Bibliotecas

In [1]:
import yfinance           as yf
import statsmodels.api    as sm
from datetime             import timedelta 
from datetime             import datetime

## 2.0 Selecionando o Ativo

In [2]:
# Lista de ativos
ativos = ["PETR4.SA", "BBAS3.SA", "BBSE3.SA", "^BVSP"]

# Selecionando a data atual
data = datetime.now()

# Buscar o histórico de 5 anos
cinco_anos_passado = data - timedelta(days = (365 * 5))

# Buscando os dados de fechamento do dia das cotações, selecionando o valor Ajustado 'Adj' 
dados_cotacoes = yf.download(ativos, cinco_anos_passado, data)['Adj Close']

dados_cotacoes

[*********************100%***********************]  4 of 4 completed


,BBAS3.SA,BBSE3.SA,PETR4.SA,^BVSP
Date,,,,
2018-06-01,22.659044,17.403339,6.350525,77240.0
2018-06-04,22.984224,17.449411,6.888905,78596.0
2018-06-05,21.520912,16.784611,6.519506,76642.0
2018-06-06,20.456694,17.014988,6.417333,76117.0
2018-06-07,19.636353,16.626640,6.193335,73851.0
...,...,...,...,...
2023-05-24,44.610001,31.370001,26.650000,108800.0
2023-05-25,44.660000,31.809999,26.450001,110054.0
2023-05-26,44.400002,31.900000,26.799999,110906.0


## 3.0 Caculando os retornos diários

In [5]:
# Calculando a variação percentual da cotação de um dia para o subsequente. A primeira linha ficará em branco, 
# pois não tem valor anterior
retornos_diarios = dados_cotacoes.pct_change().dropna()

# Mostrando a variação
retornos_diarios

,BBAS3.SA,BBSE3.SA,PETR4.SA,^BVSP
Date,,,,
2018-06-04,0.014351,0.002647,0.084777,0.017556
2018-06-05,-0.063666,-0.038099,-0.053622,-0.024861
2018-06-06,-0.049450,0.013726,-0.015672,-0.006850
2018-06-07,-0.040101,-0.022824,-0.034905,-0.029770
2018-06-08,0.014302,0.007917,-0.032360,-0.012309
...,...,...,...,...
2023-05-24,0.002923,0.019500,0.015238,-0.010270
2023-05-25,0.001121,0.014026,-0.007505,0.011526
2023-05-26,-0.005822,0.002829,0.013232,0.007742


## 4.0 Calculando o Beta

In [7]:
# Selecionando o valor de refrência que, no caso, é o índice Ibovespa | Variável independente
X = retornos_diarios["^BVSP"]

# Selcionando o ativo para calcular o Beta | aqui selecionando o primeiro ativo
Y = retornos_diarios[ativos[0]]

# Definindo o índice ibovespa como a constante da função
X = sm.add_constant(X)

# Criando o modelo com o Stats Models calculando a regressão linear e os mínimos múltiplos quadrados
model = sm.OLS(Y, X).fit()

### 4.1 Exibindo os resultados

In [14]:
print(f'O valor do BETA é: {model.params[1].round(4)}')
print(f'O valor do R²   é: {model.rsquared.round(4)}')
print("-"*78)
print(str(model.summary()))

O valor do BETA é: 1.3355
O valor do R²   é: 0.6026
------------------------------------------------------------------------------
                            OLS Regression Results                            
Dep. Variable:               PETR4.SA   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     1873.
Date:                Wed, 31 May 2023   Prob (F-statistic):          9.45e-250
Time:                        08:10:11   Log-Likelihood:                 3179.2
No. Observations:                1237   AIC:                            -6354.
Df Residuals:                    1235   BIC:                            -6344.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
